In [1]:
import yfinance as yf 
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.nn import Module,Sequential,ReLU, LSTM , Linear, Tanh
from torch.autograd import Variable
import numpy as np
from torch.nn import MSELoss,L1Loss
from torch.optim import Adam
import matplotlib.pyplot as plt
from statistics import mean
from math import sqrt
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import copy

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LOOKBACK = 5
HORIZON = 5

In [3]:
gps_stock = pd.read_csv(r"C:\Users\User\Desktop\thesis_final\stock_price_and_news\stock_news\news_dataset\final_dataset\tsla_lstm_new.csv")
print(gps_stock.shape)

(6826, 3)


In [4]:
gps_stock.drop(columns=['Date'], inplace = True)
gps_stock = gps_stock.values
print(gps_stock.shape)

(6826, 2)


In [5]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.9 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [6]:
def split_data(lookback,horizon, price):

    # (3000,2)

    data_initial = price
    window_size = lookback + horizon
    data = []
    for index in range(len(data_initial) - window_size):
        data.append(data_initial[index:index+ window_size])
    data = np.array(data) 
    # print(data.shape)
    # (300,6,2)

    # test_size = int(np.round(0.2*data.shape[0]))
    # train_set_size = data.shape[0] - (test_size)
    train_set = data
    
    x_train = train_set[:,:lookback,:]
    # print(x_train.shape)
    

    y_train = train_set[:,lookback:lookback + horizon, 1]
    y_train = y_train.reshape(-1,horizon)
    # print(y_train.shape)

    # test_set = data[train_set_size:]
    # x_test = test_set[:,:lookback]
    # y_test = test_set[:,lookback:lookback+ horizon]
    # y_test = y_test.reshape(-1,horizon)
    
    #x_train = x_train.reshape(x_train.shape[0],x_train.shape[2],x_train.shape[1])
    #y_train = y_train.reshape(y_train.shape[0],y_train.shape[2],y_train.shape[1])
    # x_test = x_test.reshape(x_test.shape[0],x_test.shape[2],x_test.shape[1])
    #y_test = y_test.reshape(y_test.shape[0],y_test.shape[2],y_test.shape[1])


    return [x_train, y_train]

In [7]:
class cnn_dataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y 
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [8]:
device = 'cpu'

In [9]:
from torch.nn import functional as F
from torch import nn

class BiLSTM_self_attention(nn.Module):
    '''
    Expected Input Shape: (batch, seq_len, channels)
    '''
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, bidirectional, dropout=0):
        super(BiLSTM_self_attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        if bidirectional :
            self.n_direction = 2
            
        self.lstm       = nn.LSTM(input_dim, 
                                  hidden_dim, 
                                  num_layers, 
                                  bidirectional=bidirectional, 
                                  dropout=dropout, 
                                  batch_first=True
                                 )
        self.fc      = nn.Linear( hidden_dim  * self.n_direction , output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        self.relu = nn.ReLU()
        self.lin_Q = nn.Linear(self.hidden_dim * self.n_direction, self.hidden_dim * self.n_direction)
        self.lin_K = nn.Linear(self.hidden_dim * self.n_direction, self.hidden_dim * self.n_direction)
        self.lin_V = nn.Linear(self.hidden_dim * self.n_direction, self.hidden_dim * self.n_direction)

    def self_attention_net(self, lstm_output ):
        q = self.lin_Q(torch.clone(lstm_output))
        k = self.lin_K(torch.clone(lstm_output))
        v = self.lin_V(torch.clone(lstm_output))
        
        attn_w = torch.bmm(q, k.transpose(1, 2))
        sfmx_attn_w = self.softmax(attn_w)
        context = torch.bmm(sfmx_attn_w, v)

        return torch.mean(context, dim=1)
  
        
    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers * self.n_direction , x.size(0), self.hidden_dim).to(device).float()
        c0 = torch.zeros(self.num_layers * self.n_direction , x.size(0), self.hidden_dim).to(device).float()
       
        # Forward propagate LSTM
        out, (_, _) = self.lstm(x, (h0, c0)) # out.shape : tensor of shape (batch_size, seq_length, hidden_size)

        context = self.self_attention_net(out)
    
        return self.fc( context  )

In [10]:
train_losses = []
valid_losses = []

In [11]:
def Train(data_loader, model,optimizer, criterion,) :
    running_loss = 0
    
    for (X,y) in (data_loader):
        X = X.to(device)
        y = y.to(device)
       
        #l2_lambda = 0.001
        #l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        #loss = loss + l2_lambda *l2_norm
        optimizer.zero_grad()
        preds = model(X.float())
        loss = criterion(preds, y.float())
       
        
        loss.backward()
        optimizer.step()
        running_loss += np.sqrt(loss.item())

    train_loss = running_loss/len(data_loader)
    # print(f'train_loss{train_loss}')

    return train_loss, model

    



def Valid(data_loader,  model, optimizer, criterion):
    """
    return y_true, y_hat
    """
    running_loss = 0
    
    
    y_hat = []
    y_true = []
    with torch.no_grad():
        for (X, y) in (data_loader):
            X = X.to(device)
            y = y.to(device)
         
            y_true = [*y_true,*(y.reshape(-1).tolist()) ]
            optimizer.zero_grad()
            preds = model(X.float())
         
            y_hat = [*y_hat, *(preds.reshape(-1).tolist())]
      
            loss = criterion(preds,y.float())
            running_loss += np.sqrt(loss.item())
            
        valid_loss = running_loss/len(data_loader)
        
        # print(f'valid_loss {valid_loss}')

    return y_true, y_hat, valid_loss, model


In [12]:

spliter = BlockingTimeSeriesSplit(n_splits = 10)

cv_test_losses = []
cv_train_losses = []
cv_val_losses = []

for trainval_idx ,test_idx in spliter.split(gps_stock):
    # print(trainval_idx)
    # print(test_idx)

    train_idx = trainval_idx[ : int(len(trainval_idx)*0.8)]
    val_idx   = trainval_idx[int(len(trainval_idx)*0.8) : ]

    x_train_unsplit = gps_stock[train_idx,:]
    x_val_unsplit  = gps_stock[val_idx,:]
    x_test_unsplit = gps_stock[test_idx,:]

    # print(x_train_unsplit.shape)
    # adadasdas
    # print(train_idx)
    # print(val_idx)

    x_train_gps, y_train_gps = split_data(LOOKBACK, HORIZON, x_train_unsplit)
    x_val_gps, y_val_gps = split_data(LOOKBACK, HORIZON, x_val_unsplit)
    x_test_gps, y_test_gps = split_data(LOOKBACK, HORIZON, x_test_unsplit)

    # print(x_train_gps.shape)
    # print(y_train_gps.shape)

    # print(x_val_gps.shape)
    # print(y_val_gps.shape)

    # print(x_test_gps.shape)
    # print(y_test_gps.shape)


    train_dataset_gps = cnn_dataset(x_train_gps,y_train_gps)
    train_loader_gps = torch.utils.data.DataLoader(train_dataset_gps, batch_size =128 , shuffle = False)
    test_dataset_gps = cnn_dataset(x_test_gps,y_test_gps)
    test_loader_gps = torch.utils.data.DataLoader(test_dataset_gps,batch_size=128,shuffle=False)
    valid_dataset_gps = cnn_dataset(x_val_gps,y_val_gps)
    valid_loader_gps = torch.utils.data.DataLoader(valid_dataset_gps,batch_size=128,shuffle=False)
    # print(len(train_dataset_gps))
    
    model = BiLSTM_self_attention(input_dim= 2, hidden_dim= 64 , num_layers= 2 , output_dim= HORIZON, bidirectional = True)
    model = model.to(device)
    criterion = MSELoss(reduction= 'mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4, weight_decay= 1e-15)

    train_losses = []
    val_losses = []

    epochs = 200

    best_val_loss = 1e+9

    for epoch in range(epochs):
        # print(epoch)
        # print('epochs {}/{}'.format(epoch+1,epochs))
        train_loss , model               = Train(train_loader_gps, model, optimizer, criterion)
        y_true, y_hat, valid_loss , model = Valid(valid_loader_gps, model, optimizer, criterion)

        # print(train_loss)
        train_losses.append(train_loss)
        val_losses.append(valid_loss)

        curr_val_loss = valid_loss

        if curr_val_loss < best_val_loss:
            best_model = copy.deepcopy(model)
            best_val_loss = curr_val_loss
            print("Best model : ", epoch)

    
    y_test_true, y_test_hat, test_loss , _ = Valid(test_loader_gps, best_model, optimizer, criterion)

    print(test_loss)
    cv_test_losses.append(test_loss)
    cv_train_losses.append(train_losses)
    cv_val_losses.append(val_losses)
    # break

print(cv_test_losses)
print(sum(cv_test_losses) / len(cv_test_losses))

Best model :  0
Best model :  1
Best model :  5
Best model :  9
Best model :  12
Best model :  17
Best model :  18
Best model :  19
Best model :  120
Best model :  121
Best model :  122
Best model :  123
Best model :  124
Best model :  125
Best model :  126
Best model :  127
Best model :  128
Best model :  129
Best model :  130
Best model :  131
Best model :  132
Best model :  133
Best model :  134
Best model :  135
Best model :  136
Best model :  137
Best model :  138
Best model :  139
Best model :  140
Best model :  141
Best model :  142
Best model :  143
Best model :  144
Best model :  145
Best model :  146
Best model :  147
Best model :  148
Best model :  149
Best model :  150
Best model :  151
Best model :  152
Best model :  153
Best model :  154
Best model :  155
Best model :  156
Best model :  157
Best model :  158
Best model :  159
Best model :  160
Best model :  161
Best model :  162
Best model :  163
Best model :  164
Best model :  165
Best model :  166
Best model :  167
Best